# Making a playable chess game with numpy and python

In [1]:
import numpy as np

We shall represent the pieces by a 2 digit number:

The first number repsents the piece type.
Pawn = 1
Rook = 2
Knight = 3
Bishop = 4
Queen = 5
King = 6

The second represents if the piece is black or white. 
0 = White
1 = Black

e.g 
10 - is a white Pawn.
61 - is a black King

In [2]:
def create_board():
    #We shall represent the board as an 8x8 numpy array
    Board = np.zeros((8, 8),  dtype=np.int) #np.int removes decimal point
    
    # Now we specify the starting position 
    #Black's pieces 
    Board[1][0:] = 11
    Board[0][0] = 21
    Board[0][1] = 31
    Board[0][2] = 41
    Board[0][3] = 51
    Board[0][4] = 61
    Board[0][5] = 41
    Board[0][6] = 31
    Board[0][7] = 21

    #White's pieces 
    Board[6][0:] = 10
    Board[7][0] = 20
    Board[7][1] = 30
    Board[7][2] = 40
    Board[7][3] = 50
    Board[7][4] = 60
    Board[7][5] = 40
    Board[7][6] = 30
    Board[7][7] = 20
    
    return Board

In [3]:
Board = create_board() 

# Here we stop nonsense moves


In [4]:
#Here we create a function to check input values are allowable

        
def allowable_input(initial_square, target_square):
    
    numbers = [str(i) for i in range(8)]
    Allowable_squares = []
    for i in numbers:
        for j in numbers:
            Allowable_squares.append(i+j)
    
    
    if initial_square not in Allowable_squares:
        return False
    if target_square not in Allowable_squares:
        return False
    else:
        return True

In [5]:
#Checking a piece has been selected
def piece_selected(initial_square,
                   target_square, 
                   chess_position):
    
    if chess_position[int(initial_square[0])][int(initial_square[1])] == 0:
        return False
    else:
        return True  

In [6]:
#checking player isnt attempting to take their own piece
def no_self_sacrifice(initial_square,
                      target_square, 
                      chess_position):
    
    if chess_position[int(target_square[0])][int(target_square[1])] ==0:
        return True
    else:
        initial_square_colour = str(chess_position[int(initial_square[0])]\
                                    [int(initial_square[1])])[1]
        target_square_colour = str(chess_position[int(target_square[0])]\
                                   [int(target_square[1])])[1]
    
        if initial_square_colour == target_square_colour:
            return False
        else:
            return True

# We we start to define how pieces move

In [7]:
"""
I shall Group pieces together from easiest to hardest to write, depending on how they move.


GROUP 1: Depends only on immediate position, and other pieces do no influence how it moves.

- Knight, although the knight jumps around the board, it has no special rules, indeed it can move to any square
as long as not occupied by one of its own. 

- Bishop, Queen - These pieces also have no special rules however, they are contrained by their own pieces as they can't
jump unlie the knight

-  Rook - Although the rook can castle, we shall consider that a move the king initiates rather than the rook

GROUP 2: Depends on the move before:

- Pawn - Be wary of en passant! - Also have to consider promotions

GROUP 3: Depends on how pieces have moved throughout entire game.

- King,- These pieces can move in special ways via castling, both king and queenside, so we have to 
create functions that allow us to check if castling is legal, on either side: 1) King and Knight can't have already move. 
2)You can't castle into or through check. It seems best to write these pieces last as these moves depend and can be 
influnced by every other piece. Note, movement rules do not depend on position but rather if thay have already moved.

The King is the most complex pieces.
"""

"\nI shall Group pieces together from easiest to hardest to write, depending on how they move.\n\n\nGROUP 1: Depends only on immediate position, and other pieces do no influence how it moves.\n\n- Knight, although the knight jumps around the board, it has no special rules, indeed it can move to any square\nas long as not occupied by one of its own. \n\n- Bishop, Queen - These pieces also have no special rules however, they are contrained by their own pieces as they can't\njump unlie the knight\n\n-  Rook - Although the rook can castle, we shall consider that a move the king initiates rather than the rook\n\nGROUP 2: Depends on the move before:\n\n- Pawn - Be wary of en passant! - Also have to consider promotions\n\nGROUP 3: Depends on how pieces have moved throughout entire game.\n\n- King,- These pieces can move in special ways via castling, both king and queenside, so we have to \ncreate functions that allow us to check if castling is legal, on either side: 1) King and Knight can't h

# Knight

In [8]:
"""
In essence whatever the starting position of white, we either add or subtract 1 or 2 to each square value
If any are not on (0,7) then we remove them as they are not on the board
If they land on their own piece its not allowed
"""
def possible_knight_positions(initial_square, #Note enter initial_square as a str
                              chess_position): 
    
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    #Possible knights moves
    x_addition = np.array([-2,-2,-1,-1,1,1,2,2])
    y_addition = np.array([1,-1,2,-2,2,-2,1,-1])
    possible_x = x_addition + xlabel
    possible_y = y_addition + ylabel
    
    tuple_of_possible_positions = list(zip(possible_x,possible_y))
    
    
    #Removing tuples not on the board
    acceptable_values = [0,1,2,3,4,5,6,7]
    result1 = [i for i in tuple_of_possible_positions if i[0] in acceptable_values]
    result2 = [i for i in result1 if i[1] in acceptable_values]
    
    
    #Removing tuples that represent taking own piece
    result3 = []
    for i in result2:
        string_of_i = str(i[0]) + str(i[1])
        if no_self_sacrifice(initial_square, string_of_i, chess_position) == True:
            result3.append(i)
    
   
    return result3

# Bishop

In [9]:
"""
The Bishop can move in 4 directions, as bishop traverses a diagonal the index increments either:
We call them:
UR: (-1,1)  #Upper Right
UL: (-1,-1) #Upper Left
LL: (1,-1)  #Lower Left
LR: (1,1)   #Lower Right
"""
def possible_bishop_positions(initial_square,chess_position):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
 
    
    #First we check the possibilities in UR (-1,1) 
    new_xlabel = xlabel
    new_ylabel = ylabel
    
    while True:
        new_xlabel -= 1
        new_ylabel += 1
        if new_xlabel < 0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break   
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in UL (-1,-1)
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_xlabel -= 1
        new_ylabel -= 1
        if new_xlabel < 0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break   
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    
    
    #Then we check the possibilities in LL (1,-1)
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_xlabel += 1
        new_ylabel -= 1
        if new_xlabel < 0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break  
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in LR (1,1)
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        new_xlabel += 1
        new_ylabel += 1
        if new_xlabel <0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break   
        
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    return tuple_of_possible_positions
    
    
    

# Rook


In [10]:
"""
The Rook can move in 4 directions:
We call them:
U: (-1,0)  #Upper Right
D: (1,0) #Upper Left
L: (0,-1)  #Lower Left
R: (0,1)   #Lower Right
"""
def possible_rook_positions(initial_square,
                            chess_position):
    
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
 
    
    #First we check the possibilities in U: (-1,0) #Upper Right
    new_xlabel = xlabel
    new_ylabel = ylabel
    
    while True:
        new_xlabel -= 1
        
        if new_xlabel < 0:
            break
       
        if new_xlabel >7:
            break
         
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in D: (1,0) #Upper Left
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_xlabel += 1
        
        if new_xlabel < 0:
            break
        
        if new_xlabel >7:
            break
       
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    
    
    #Then we check the possibilities in L: (0,-1)  #Lower Left
    new_xlabel = xlabel
    new_ylabel = ylabel
    
    while True:
        new_ylabel -= 1
        
        if new_ylabel <0:
            break
        
        if new_ylabel >7:
            break  
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in R: (0,1) #Lower Right
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_ylabel += 1
        
        if new_ylabel <0:
            break
            
        if new_ylabel >7:
            break   
        
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] !=\
        str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    return tuple_of_possible_positions

# Queen

In [11]:
#The Queen moves the same as a rook and a bishop
def possible_queen_positions(initial_square,
                             chess_position):
    
    possible_queen_positions = possible_rook_positions(initial_square,chess_position) + \
    possible_bishop_positions(initial_square,chess_position)
    
    return possible_queen_positions

# White Pawns


In [12]:
#To account for enpassant, we shall 
def enpassant():
    if True:
        return 'y'
    if False:
        return False

In [13]:
if 3 > 1 >=1:
    print(True)

True


In [14]:
#Here the direction Matters
def possible_white_pawn_positions(initial_square,
                                  chess_position, 
                                  enpassant = False, 
                                  Promotion = 'Queen'):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    
    tuple_of_possible_positions = []
    #Pawn can move one or two places if it is yet to move
    if xlabel == 6:
        if chess_position[xlabel -1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel-1,ylabel))
            if chess_position[xlabel -2 ][ylabel] ==0:
                tuple_of_possible_positions.append((xlabel-2,ylabel))
            
    #If pawn has moved it can only move one piece forward
    if xlabel != 6:
        if chess_position[xlabel -1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel-1,ylabel))

                                  

    #Capturing pieces forward left                                                                                  
    if 7>= xlabel -1 >= 0:
        if 7>= ylabel -1 >=0:
            
            if chess_position[xlabel -1][ylabel-1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel -1 ][ylabel -1])[1]:
                    tuple_of_possible_positions.append((xlabel-1,ylabel-1))
           
        
    #Capturing pieces forward right                                                                                  
    if 7>= xlabel -1 >= 0:
        if 7>= ylabel +1 >=0:
            if chess_position[xlabel -1][ylabel+1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel -1 ][ylabel +1])[1]:
                    tuple_of_possible_positions.append((xlabel-1,ylabel+1))                                      
    
    #EN-Passant 
    #Note to determine if enpassant is permetting i will have a function enpassant, 
    #either outputting False or the rank permitted                                          
    if xlabel == 3:
        if enpassant != False:
            tuple_of_possible_positions.append((2,enpassant()))
    return tuple_of_possible_positions 

In [15]:
possible_white_pawn_positions('63',Board)

[(5, 3), (4, 3)]

# Black pawns

In [16]:
def possible_black_pawn_positions(initial_square,
                                  chess_position,
                                  enpassant = False,
                                  Promotion = 'Queen'):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    
    #Start we white pawns
    tuple_of_possible_positions = []
    #Pawn can move one or two places if it is yet to move
    if xlabel == 1:
        if chess_position[xlabel +1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel+1,ylabel))
            if chess_position[xlabel +2 ][ylabel] ==0:
                tuple_of_possible_positions.append((xlabel+2,ylabel))
    ####
    #If pawn has moved it can only move one piece forward
    if xlabel != 1:
        if chess_position[xlabel +1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel+1,ylabel))
    
    #Capturing pieces forward left                                                                                  
    if 7>= xlabel +1 >= 0:
        if 7>= ylabel +1 >=0:
            if chess_position[xlabel +1 ][ylabel+1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel +1 ][ylabel +1])[1]:
                    tuple_of_possible_positions.append((xlabel-1,ylabel-1))
            
        
    #Capturing pieces forward right                                                                                  
    if 7>= xlabel +1 >= 0:
        if 7>= ylabel -1 >=0:
            if chess_position[xlabel +1 ][ylabel-1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel +1 ][ylabel -1])[1]:
                    tuple_of_possible_positions.append((xlabel+1,ylabel-1))                                      
    
                                               
  
        
    return tuple_of_possible_positions    

# We write a series of functions outputting squares attacked by all a colors pieces

In [17]:
#We find all locations of pieces of a certain type

In [18]:
def location_of_certain_piece(chess_position,
                    color,# 'White' or 'Black'
                    piece # 'Pawn', 'Rook', 'Knight','Bishop','Queen','King'
                    ): 
    
    if color == 'White':
        if piece == 'Pawn':
            specific_piece = 10
        if piece == 'Rook':
            specific_piece = 20
        if piece == 'Knight':
            specific_piece = 30
        if piece == 'Bishop':
            specific_piece = 40
        if piece == 'Queen':
            specific_piece = 50
        if piece == 'King':
            specific_piece = 60
            
    
    if color == 'Black':
        if piece == 'Pawn':
            specific_piece = 11
        if piece == 'Rook':
            specific_piece = 21
        if piece == 'Knight':
            specific_piece = 31
        if piece == 'Bishop':
            specific_piece = 41
        if piece == 'Queen':
            specific_piece = 51
        if piece == 'King':
            specific_piece = 61
    
    #Getting locations
    locations = np.where(chess_position == specific_piece)
    #We convert numpy array to tuple
    locations = tuple(map(tuple, locations))
    #Note numpy gives us two arrays of x and y coordinates
    #We want to convert this to a list of x and ys paired together
    
    x = list(locations[0])
    y = list(locations[1])
    list_of_tuples_of_locations = list(tuple(zip(x,y)))
    return list_of_tuples_of_locations

# Squares Attacked

In [19]:
def convert_list_of_tuples_to_list_of_strings(list_of_tuples):
    list_of_strings = []
    for i in list_of_tuples:
        string = str(i[0]) + str(i[1])
        list_of_strings.append(string)
    return list_of_strings

In [20]:
def possible_naive_King_positions(initial_square, chess_position):
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    #Possible Kings moves
    x_addition = np.array([-1,-1,-1,0,0,1,1,1])
    y_addition = np.array([-1,0,1,-1,1,-1,0,+1])
    possible_x = x_addition + xlabel
    possible_y = y_addition + ylabel
    
    tuple_of_possible_positions = list(zip(possible_x,possible_y))
    #Removing tuples not on the board
    acceptable_values = [0,1,2,3,4,5,6,7]
    result1 = [i for i in tuple_of_possible_positions if i[0] in acceptable_values]
    result2 = [i for i in result1 if i[1] in acceptable_values]
    
    
    #Removing tuples that represent taking own piece
    result3 = []
    for i in result2:
        string_of_i = str(i[0]) + str(i[1])
        if no_self_sacrifice(initial_square, string_of_i, chess_position) == True:
            result3.append(i)
    
   
    return result3


As a precursor to creating a function that generates all squares attacked by a certain color, we write a function that generates pieces attacked by pawns. As pawns are the only piece that can move to a square without the possibility of capturing a piece on that sqaure. e.g. pawns moving directly forwards. Note en passant is not included in attacking squares, as en passant only takes other pawns and doesnt attack the king!

In [21]:
#Here the direction Matters
def squares_white_pawns_attack(initial_square,
                               chess_position
                              ):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
 
    tuple_of_possible_positions = []

    #Capturing pieces forward left                                                                                  
    if 7>= xlabel -1 >= 0:
        if 7>= ylabel -1 >=0:
            if chess_position[xlabel -1][ylabel-1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel -1 ][ylabel -1])[1]:
                    tuple_of_possible_positions.append((xlabel-1,ylabel-1))

    #Capturing pieces forward right                                                                                  
    if 7>= xlabel -1 >= 0:
        if 7>= ylabel +1 >=0:
            if chess_position[xlabel -1][ylabel+1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel -1 ][ylabel +1])[1]:
                    tuple_of_possible_positions.append((xlabel-1,ylabel+1))
    
     
    return tuple_of_possible_positions 
    
   

In [22]:
def squares_black_pawns_attack(initial_square,
                                  chess_position,
                                  ):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    tuple_of_possible_positions = []
    
    #Capturing pieces forward left                                                                                  
    if 7>= xlabel +1 >= 0:
        if 7>= ylabel +1 >=0:
            if chess_position[xlabel +1 ][ylabel+1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel +1 ][ylabel +1])[1]:
                    tuple_of_possible_positions.append((xlabel-1,ylabel-1))
            
        
    #Capturing pieces forward right                                                                                  
    if 7>= xlabel +1 >= 0:
        if 7>= ylabel -1 >=0:
            if chess_position[xlabel +1 ][ylabel-1] != 0:
                if str(chess_position[xlabel][ylabel])[1] !=\
                str(chess_position[xlabel +1 ][ylabel -1])[1]:
                    tuple_of_possible_positions.append((xlabel+1,ylabel-1))                                      
    
                                               
  
        
    return tuple_of_possible_positions  

In [23]:
def squares_attacked_by_side(chess_position,
            color,# 'White' or 'Black'
            piece # 'Pawn', 'Rook', 'Knight','Bishop','Queen','King'
                    ): 
    
                           
    locations = location_of_certain_piece(chess_position,color,piece)
    
    locations = convert_list_of_tuples_to_list_of_strings(locations)
    Attacked_Squares = []
       
        
    if piece == 'Rook':
        for i in locations:
            squares = possible_rook_positions(i, chess_position)
            Attacked_Squares.append(squares)
            
          
    if piece == 'Knight':
        for i in locations:
            squares = possible_knight_positions(i,chess_position)
            Attacked_Squares.append(squares)
        
        
    if piece == 'Bishop':
        for i in locations:
            squares = possible_bishop_positions(i,chess_position)
            Attacked_Squares.append(squares)
        
    if piece == 'Queen':
        for i in locations:
            squares = possible_queen_positions(i,chess_position)
            Attacked_Squares.append(squares)
        
    if piece == 'King':
        for i in locations:
            squares = possible_naive_King_positions(i,chess_position)
            Attacked_Squares.append(squares)
    
    if color == 'White':
        if piece == 'Pawn':
            for i in locations:
                squares = squares_white_pawns_attack(i, chess_position)
                Attacked_Squares.append(squares)
    if color == 'Black':
        if piece == 'Pawn':
            for i in locations:
                squares = squares_black_pawns_attack(i, chess_position)
                Attacked_Squares.append(squares)
        
    #Attacked_Squares is a list of lists
    #We convert to a flatlist
    Attacked_Squares = [item for sublist in Attacked_Squares for item in sublist]
    #Remove multiples
    Attacked_Squares = list(set(Attacked_Squares))
    return Attacked_Squares

In [24]:
def Squares_White_Attacks(chess_position):
    pieces = ['Pawn', 'Rook', 'Knight','Bishop','Queen','King']
    Squares_White_Attacks = []
    for piece in pieces:
        Squares_White_Attacks.append(squares_attacked_by_side(chess_position,'White',piece))
    
    
    #We convert to a flatlist
    Squares_White_Attacks = [item for sublist in Squares_White_Attacks for item in sublist]
    #Remove multiples
    Squares_White_Attacks = list(set(Squares_White_Attacks))
    return Squares_White_Attacks
    

In [25]:
Squares_White_Attacks(Board)

[(5, 7), (5, 2), (5, 5), (5, 0)]

In [26]:
def Squares_Black_Attacks(chess_position):
    pieces = ['Pawn', 'Rook', 'Knight','Bishop','Queen','King']
    Squares_Black_Attacks = []
    for piece in pieces:
        Squares_Black_Attacks.append(squares_attacked_by_side(chess_position,'Black',piece))
    
    
    #We convert to a flatlist
    Squares_Black_Attacks = [item for sublist in Squares_Black_Attacks for item in sublist]
    #Remove multiples
    Squares_Black_Attacks = list(set(Squares_Black_Attacks))
    return Squares_Black_Attacks

In [27]:
#We now create a function that outputs all possible moves for white
def all_possible_moves_white(chess_position):
    all_moves = Squares_White_Attacks(chess_position)
    
    locations = location_of_certain_piece(chess_position,'White','Pawn')
    
    locations = convert_list_of_tuples_to_list_of_strings(locations)
    
    mylist = []
    
    for i in locations:
        squares = possible_white_pawn_positions(i, chess_position)
        
        #print(squares)
        
        mylist.append(squares)
        
    all_moves = mylist + [all_moves]    
    
    #print(all_moves)
    #We convert to a flatlist
    all_moves = [item for sublist in all_moves for item in sublist]
    #print(all_moves)
    #Remove multiples
    all_moves = list(set(all_moves))
    return all_moves   

In [28]:
#We now create a function that outputs all possible moves for black


#We now create a function that outputs all possible moves for white
def all_possible_moves_black(chess_position):
    all_moves = Squares_Black_Attacks(chess_position)
    
    locations = location_of_certain_piece(chess_position,'Black','Pawn')
    
    locations = convert_list_of_tuples_to_list_of_strings(locations)
    
    mylist = []
    
    for i in locations:
        squares = possible_black_pawn_positions(i, chess_position)
        
        #print(squares)
        
        mylist.append(squares)
        
    all_moves = mylist + [all_moves]    
    
    #print(all_moves)
    #We convert to a flatlist
    all_moves = [item for sublist in all_moves for item in sublist]
    #print(all_moves)
    #Remove multiples
    all_moves = list(set(all_moves))
    return all_moves  

# Now we are ready to define how king moves

In [29]:
def possible_White_King_positions(initial_square, chess_position, White_Queen_Side_Castle = False, White_King_Side_Castle = False):
    
    positions = possible_naive_King_positions(initial_square,chess_position)
    
    #We remove positions that correspond to moving into check
    
    Squares_Black_Attacks = Squares_Black_Attacks(chess_position)
    
    for square in Squares_Black_Attacks:
        if square in positions:
            positions.remove(square)
    
    
    if White_Queen_Side_Castle == True:
        positions.append((7,2))
    if White_King_Side_Castle == True:
        positions.append((7,6))
        
    
    return positions
    

In [30]:
def possible_Black_King_positions(initial_square, 
                        chess_position, 
                        Black_Queen_Side_Castle = False, 
                        Black_King_Side_Castle = False):
    
    positions = possible_naive_King_positions(initial_square,chess_position)
    
    #We remove positions that correspond to moving into check
    
    Squares_White_Attacks = Squares_White_Attacks(chess_position)
    
    for square in Squares_White_Attacks:
        if square in positions:
            positions.remove(square)
    
    if Black_Queen_Side_Castle == True:
        positions.append((0,2))
    if Black_King_Side_Castle == True:
        positions.append((0,6))  

# Is White in Check?

In [31]:
def is_white_in_check(chess_position):
    
    white_king = location_of_certain_piece(chess_position,'White','King')
    
    
    #UnboundLocalError: local variable 'Squares_Black_Attacks' referenced before assignment
    #To fix error i changed Squares_Black_Attacks to Squares_Black_Attacks_
    Squares_Black_Attacks_ = Squares_Black_Attacks(chess_position)
    

    if white_king in Squares_Black_Attacks_:
        return True
    else:
        return False

# Is Black in Check?

In [50]:
def is_black_in_check(chess_position):
    
    black_king = location_of_certain_piece(chess_position,'Black','King')
    
    
    #UnboundLocalError: local variable 'Squares_Black_Attacks' referenced before assignment
    #To fix error i changed Squares_Black_Attacks to Squares_Black_Attacks_
    Squares_White_Attacks_ = Squares_White_Attacks(chess_position)
    

    if white_king in Squares_White_Attacks_:
        return True
    else:
        return False

In [ ]:
#First we write a function to alter our Board
def is_move_legitimate(initial_square, 
                 target_square, 
                 chess_position, 
                 Turn #White or Black
                ): 
    #Checking input is allowed
    if allowable_input(initial_square, target_square) ==False:
        return False
    
    #Checking squares selected correspond to an actual piece piece 
    if piece_selected(initial_square,target_square, chess_position)== False:
        return False
    
    #FUNCTION TO CHECK PLAYER HAS SELECTED THEIR OWN PIECE
    
    #CHECK CONDITIONS ABOVE
    
    

In [35]:
Board

array([[21, 31, 41, 51, 61, 41, 31, 21],
       [11, 11, 11, 11, 11, 11, 11, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10, 10, 10, 10, 10],
       [20, 30, 40, 50, 60, 40, 30, 20]])

In [ ]:
#First we write a function to alter our Board
def move_a_piece(initial_square, 
                 target_square, 
                 chess_position, 
                 Turn #White or Black
                ): 
 
 
    xlabel_initial = int(initial_square[0]) 
    ylabel_initial =int(initial_square[1])
    
    xlabel_target = int(target_square[0]) 
    ylabel_target =int(target_square[1])
    
    piece = Board[xlabel_initial][ylabel_initial]
    
    
    #For Any given move the initial square will always change to Zero, Hence
    Board[xlabel_initial][ylabel_initial] = 0

    #Pawn Premotion White
    if piece == 10:
        if xlabel_target ==0:
            #Promoting to Queen
            Board[xlabel_target][ylabel_target] = 50 #####NOTE Queen promotion defaulted, will change later
                    
    #Pawn Promotion Black
    if piece == 11:
        if xlabel_target ==7:
            #Promoting to Queen
            Board[xlabel_target][ylabel_target] = 51 #####NOTE Queen promotion defaulted, will change later
            
    #Specify special moves for: CASTLE Kingside Black
    if piece == 61:
        if initial_square == '04':
            if target_square == '06':
                #Moving the King
                Board[xlabel_target][ylabel_target] = piece
                #Move the Rook
                Board[0][5] = 21
                #Clear Rook
                Board[0][7] = 0

    #Specify special moves for: CASTLE Queenside Black    
    if piece == 61:
        if initial_square == '04':
            if target_square == '02':
                #Moving the King
                Board[xlabel_target][ylabel_target] = piece
                #Move the Rook
                Board[0][3] = 21
                #Clear Rook
                Board[0][0] = 0

    
     #Specify special moves for: CASTLE Kingside White
    if piece == 60:
        if initial_square == '74':
            if target_square == '76':
                #Moving the King
                Board[xlabel_target][ylabel_target] = piece
                #Move the Rook
                Board[7][5] = 20
                #Clear Rook
                Board[7][7] = 0

    #Specify special moves for: CASTLE Queenside White  
    if piece == 60:
        if initial_square == '74':
            if target_square == '72':
                #Moving the King
                Board[xlabel_target][ylabel_target] = piece
                #Move the Rook
                Board[7][3] = 20
                #Clear Rook
                Board[7][0] = 0
    
    #ENPASSANT White
    if piece == 10:
        #Check on right rank for enpassant
        if xlabel_initial ==3:
            #Check nothing on target square
            if Board[xlabel_target][xlabel_target] == 0:
                
                if ylabel_initial -1 == ylabel_target:
                    #Taking the pawn
                    Board[3][ylabel_initial -1] = 0
                    #Moving Square
                    Board[xlabel_target][ylabel_target] = piece
                
                if ylabel_initial +1 == ylabel_target:
                    #Taking the pawn
                    Board[3][ylabel_initial +1] = 0
                    #Moving Square
                    Board[xlabel_target][ylabel_target] = piece
                
    
    
    #ENPASSANT BLACK
     if piece == 11:
        #Check on right rank for enpassant
        if xlabel_initial ==4:
            #Check nothing on target square
            if Board[xlabel_target][xlabel_target] == 0:
                
                if ylabel_initial -1 == ylabel_target:
                    #Taking the pawn
                    Board[4][ylabel_initial -1] = 0
                    #Moving Square
                    Board[xlabel_target][ylabel_target] = piece
                
                if ylabel_initial +1 == ylabel_target:
                    #Taking the pawn
                    Board[4][ylabel_initial +1] = 0
                    #Moving Square
                    Board[xlabel_target][ylabel_target] = piece
    
    #REGULAR MOVES
    else:
        #Updating new Square
        Board[xlabel_target][ylabel_target] = piece

# Is White in CheckMate?

In [51]:
def is_white_in_checkmate(chess_position):
    if is_white_in_check(chess_position) == False:
        return False
    
    #Checking king can't move
        
    initial_square = location_of_certain_piece(chess_position,'White', 'King')
    initial_sqaure = convert_list_of_tuples_to_list_of_strings(initial_square)
        
    elif len(possible_naive_King_positions(initial_square, chess_position) != 0:
               return False
    
    elif      :
    elif:
               
    
    
    
    
    possible_naive_King_positions(initial_square, chess_position)

SyntaxError: invalid syntax (<ipython-input-51-62d5d5d355f7>, line 10)

# Is Black in CheckMate?

In [52]:
def is_black_in_checkmate(chess_position):
    if is_black_in_check(chess_position) == False:
        return False
    else:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-52-1e714a0b3926>, line 5)

# Is White in StaleMate?

In [53]:
def is_white_in_stalemate(chess_position):
    if is_white_in_check(chess_position) == True:
        return False
    else:
        if len(all_possible_moves_black(chess_position))== 0:
            return True  
        
        
        

# Is Black in StaleMate?

In [54]:
def is_black_in_stalemate(chess_position):
    if is_black_in_check(chess_position) == True:
        return False
    else:
        if len(all_possible_moves_black(chess_position))== 0:
            return True   
        
        
        
        
        

In [56]:
'01'[0]

'0'

In [57]:
move_a_piece('01', '02')
    

TypeError: move_a_piece() missing 1 required positional argument: 'chess_position'

In [34]:
game_over = False
turn = 0

while not game_over:
    #White moves
    
    if turn == 0:
        move == input()
    
    if turn == 0:
       
    
    #Black moves

SyntaxError: unexpected EOF while parsing (<ipython-input-34-aeafa4e0196e>, line 13)